In [1]:
#This cell reads all the 'team-year-csv' and combines them into one csv per team

import pandas as pd


teams = [
    'ATL', 'BOS', 'BRK', 'CHO', 'CHI', 'CLE', 
    'DAL', 'DEN', 'DET', 'GSW', 'HOU', 'IND', 
    'LAC', 'LAL', 'MEM', 'MIA', 'MIL', 'MIN', 
    'NOP', 'NYK', 'OKC', 'ORL', 'PHI', 'PHO',
    'POR', 'SAC', 'SAS', 'TOR', 'UTA', 'WAS'
    ]


for team in teams:    
    df1 = pd.read_csv(f'./data/{team}-21-22.csv') 
    df2 = pd.read_csv(f'./data/{team}-ADV-21-22.csv') 
    df3 = pd.read_csv(f'./data/{team}-22-23.csv') 
    df4 = pd.read_csv(f'./data/{team}-ADV-22-23.csv') 

    #Cleanup df1
    col_names = df1.iloc[0, :].to_list()

    col_names = col_names[:24]
    test = df1.iloc[:, 0:24]
    #drop first row
    test = test.drop([0])

    #rename home and away columns
    col_names[3] = "H/A"
    test.columns = col_names

    #drop unneeded columns
    test = test.drop(test.columns[[0,1]],axis = 1)

    test['H/A'] = test['H/A'].apply(lambda x: "A" if x == "@" else "H")
    test.columns = ['Date', 'H/A', 'Opp', 'W/L', 'P', 'OppP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF']
    df1 = test

    #Cleanup df2
    col_names = df2.iloc[0, :].to_list()
    test = df2

    #drop first row
    test = test.drop([0])

    #rename Home and away column
    col_names[3] = "H/A"
    test.columns = col_names

    #drop unneeded columns
    test = test.drop(test.columns[[0,1,3,4,5,6,7,18,23]],axis = 1)

    test.columns = ['Date', 'ORtg', 'DRtg', 'Pace', 'FTr', '3PAr', 'TS%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'OeFG%', 'OTOV%', 'OORB%', 'OFT/FGA', 'DeFG%', 'DTOV%', 'DDRB%', 'DFT/FGA']
    df2 = test

    #Cleanup df3
    col_names = df3.iloc[0, :].to_list()

    col_names = col_names[:24]
    test = df3.iloc[:, 0:24]
    #drop first row
    test = test.drop([0])

    #rename home and away columns
    col_names[3] = "H/A"
    test.columns = col_names

    #drop unneeded columns
    test = test.drop(test.columns[[0,1]],axis = 1)

    test['H/A'] = test['H/A'].apply(lambda x: "A" if x == "@" else "H")
    test.columns = ['Date', 'H/A', 'Opp', 'W/L', 'P', 'OppP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF']
    df3 = test

    #Cleanup df4
    col_names = df4.iloc[0, :].to_list()
    test = df4

    #drop first row
    test = test.drop([0])

    #rename Home and away column
    col_names[3] = "H/A"
    test.columns = col_names

    #drop unneeded columns
    test = test.drop(test.columns[[0,1,3,4,5,6,7,18,23]],axis = 1)

    test.columns = ['Date', 'ORtg', 'DRtg', 'Pace', 'FTr', '3PAr', 'TS%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'OeFG%', 'OTOV%', 'OORB%', 'OFT/FGA', 'DeFG%', 'DTOV%', 'DDRB%', 'DFT/FGA']
    df4 = test

    testdf = pd.merge(df1, df2, left_on='Date', right_on='Date')
    testdf2 = pd.merge(df3, df4, left_on='Date', right_on='Date')
    finaldf = pd.concat([testdf, testdf2])

    #sort final df by date
    finaldf['Date'] = pd.to_datetime(finaldf['Date'])
    finaldf = finaldf.sort_values(by='Date', ascending=True)
    finaldf.to_csv("./data/" + team + ".csv")

In [4]:
#This cell reads all the individual team.csv and creates dataset.csv
dataset_df = pd.DataFrame()

for team in teams:
    home_df = pd.read_csv('./data/' + team + '.csv')
    num_rows = home_df.shape[0]

    for i in range(50,num_rows):
        #only calculate stats if team is home team
        if home_df.iloc[i]['H/A'] == 'A':
            continue
        #get home team stats
        winner = home_df.iloc[i]['W/L'] == 'W'
        index = i
        
        #calculate average over last 3 games
        wins_last_3 = home_df.iloc[index - 3:index]['W/L'].value_counts().get('W', 0)
        home_last_3 = home_df.iloc[index - 3:index]['H/A'].value_counts().get('H', 0)
        numerical_stats_last_3 = home_df.iloc[index-3:index,5:].mean().apply(lambda x: float("{:.2f}".format(x)))
        last_3 = numerical_stats_last_3
        last_3['W'] = wins_last_3
        last_3['H'] = home_last_3
        last_3.index = list(map(lambda n: "last_3_" + n, last_3.index.to_list()))
        
        #calculate average over last 10 games
        wins_last_10 = home_df.iloc[index - 10:index]['W/L'].value_counts().get('W', 0)
        home_last_10 = home_df.iloc[index - 10:index]['H/A'].value_counts().get('H', 0)
        numerical_stats_last_10 = home_df.iloc[index-10:index,5:].mean().apply(lambda x: float("{:.2f}".format(x)))
        last_10 = numerical_stats_last_10
        last_10['W'] = wins_last_10
        last_10['H'] = home_last_10
        last_10.index = list(map(lambda n: "last_10_" + n, last_10.index.to_list()))

        #calculate average over last 50 games
        wins_last_50 = home_df.iloc[index - 50:index]['W/L'].value_counts().get('W', 0)
        home_last_50 = home_df.iloc[index - 50:index]['H/A'].value_counts().get('H', 0)
        numerical_stats_last_50 = home_df.iloc[index-50:index,5:].mean().apply(lambda x: float("{:.2f}".format(x)))
        last_50 = numerical_stats_last_50
        last_50['W'] = wins_last_50
        last_50['H'] = home_last_50
        last_50.index = list(map(lambda n: "last_50_" + n, last_50.index.to_list()))
        #add all averages to one series
        home_stats = pd.concat([last_3, last_10, last_50])
        home_stats.index = list(map(lambda n: "home_" + n, home_stats.index.to_list()))

        #get away team stats
        away_team = home_df.iloc[index]['Opp']
        date = home_df.iloc[index]['Date']
        away_df = pd.read_csv('./data/' + away_team.upper() + '.csv')
        index = pd.Index(away_df['Date']).get_loc(date)
        if index < 50: #If the away team doesn't have 50 previous games in the database then we just skip this game
            pass

        #calculate average over last 3 games
        wins_last_3 = away_df.iloc[index - 3:index]['W/L'].value_counts().get('W', 0)
        home_last_3 = away_df.iloc[index - 3:index]['H/A'].value_counts().get('H', 0)
        numerical_stats_last_3 = away_df.iloc[index-3:index,5:].mean().apply(lambda x: float("{:.2f}".format(x)))
        last_3 = numerical_stats_last_3
        last_3['W'] = wins_last_3
        last_3['H'] = home_last_3
        last_3.index = list(map(lambda n: "last_3_" + n, last_3.index.to_list()))

        #calculate average over last 10 games
        wins_last_10 = away_df.iloc[index - 10:index]['W/L'].value_counts().get('W', 0)
        home_last_10 = away_df.iloc[index - 10:index]['H/A'].value_counts().get('H', 0)
        numerical_stats_last_10 = away_df.iloc[index-10:index,5:].mean().apply(lambda x: float("{:.2f}".format(x)))
        last_10 = numerical_stats_last_10
        last_10['W'] = wins_last_10
        last_10['H'] = home_last_10
        last_10.index = list(map(lambda n: "last_10_" + n, last_10.index.to_list()))

        #calculate average over last 50 games
        wins_last_50 = away_df.iloc[index - 50:index]['W/L'].value_counts().get('W', 0)
        home_last_50 = away_df.iloc[index - 50:index]['H/A'].value_counts().get('H', 0)
        numerical_stats_last_50 = away_df.iloc[index-50:index,5:].mean().apply(lambda x: float("{:.2f}".format(x)))
        last_50 = numerical_stats_last_50
        last_50['W'] = wins_last_50
        last_50['H'] = home_last_50
        last_50.index = list(map(lambda n: "last_50_" + n, last_50.index.to_list()))
        
        #add all averages to one series
        away_stats = pd.concat([last_3, last_10, last_50])
        away_stats.index = list(map(lambda n: "away_" + n, away_stats.index.to_list()))

        #add home and away stats
        stats = pd.concat([home_stats, away_stats])
        stats['Date'] = date
        stats['Home_Team'] = team
        stats['Away_Team'] = away_team
        stats['Home_Team_Won'] = winner

        #add stats to the dataset
        dataset_df = pd.concat([dataset_df, stats.to_frame().T])


#save data
dataset_df.to_csv('./data/dataset.csv')


        
    